Just in case one ends up wanting to modify the stored VQA (e.g., once siting typos.)

This will basically pull the JSON files and re-write them in a new directory, including any modifications to VQA prompts made in the utilities w/o having to regenerate all new histograms.

In [27]:
data_in_dir = '../example_hists/' # jsons
data_out_dir = '/Users/jnaiman/Downloads/tmp/JCDL2025/example_hists/modifications/' # jsons

verbose_qa = True


In [28]:
import json
from importlib import reload
from glob import glob
import numpy as np

from sys import path
path.append('../')

import utils.plot_qa_utils
reload(utils.plot_qa_utils)
from utils.plot_qa_utils import init_qa_pairs

import utils.histogram_plot_qa_utils
reload(utils.histogram_plot_qa_utils)
from utils.histogram_plot_qa_utils import q_nbars_hist_plot_plotnums, q_stats_hists, q_gmm_ngaussians_hists

from utils.data_utils import NumpyEncoder

In [29]:
stats = {'minimum':np.min, 'maximum':np.max, 'median':np.median, 'mean':np.mean}

In [30]:
jsons_in = glob(data_in_dir + '/jsons/*.json')
jsons_in[:3]

['../example_hists//jsons/id_0077.json',
 '../example_hists//jsons/id_0020.json',
 '../example_hists//jsons/id_0036.json']

In [32]:
for json_in in jsons_in:
    with open(json_in,'r') as f:
        j = json.load(f)
        datas_in = json.loads(j)

    # fake "axes" with plot numbers
    axes = []
    for k,v in datas_in.items():
        if 'plot' in k:
            axes.append([])

    datas = datas_in.copy()
    ######## GENERATE QA BASE #######
    qa_pairs = init_qa_pairs()
    for iplot in range(len((axes))):
        if datas['plot'+str(iplot)]['type'] == 'histogram':
            ############ L1 #############
            # number of bars
            if len(axes) > 1:
                qa_pairs = q_nbars_hist_plot_plotnums(datas, qa_pairs, plot_num = iplot, use_words=False, verbose=verbose_qa)
            qa_pairs = q_nbars_hist_plot_plotnums(datas, qa_pairs, plot_num = iplot, use_words=True, verbose=verbose_qa)

        ###### L2 #######
        # stats items
        for k,v in stats.items():
            if len(axes) > 1:
                qa_pairs = q_stats_hists(datas, qa_pairs, stat={k:v}, plot_num=iplot, use_words=False, verbose=verbose_qa)
            qa_pairs = q_stats_hists(datas, qa_pairs, stat={k:v}, plot_num=iplot, use_words=True, verbose=verbose_qa)

            ###### L3 ######
            # if GMM -- how many gaussians?
            hasGMM = False
            if 'data params' not in datas['plot'+str(iplot)]['data']:
                #print('Not a gmm relationship!')
                pass
            else:
                if datas['plot'+str(iplot)]['distribution'] == 'gmm':
                    hasGMM = True
            if hasGMM:       
                qa_pairs = q_gmm_ngaussians_hists(datas, qa_pairs, plot_num=iplot, use_words=True, verbose=verbose_qa)

    datas['VQA'] = qa_pairs

    # dump full data
    dumped = json.dumps(datas, cls=NumpyEncoder)
    with open(data_out_dir + 'jsons/' + json_in.split('/')[-1], 'w') as f:
        json.dump(dumped, f)

QUESTION: You are a helpful assistant that can analyze images.  How many bars are there in the specified figure panel? Please format the output as a json as {"nbars":""} for this figure panel, where the "nbars" value should be an integer.
ANSWER: {'nbars': 20}
QUESTION: You are a helpful assistant that can analyze images.  What is the minimum value of the data in this figure panel?  Please format the output as a json as {"minimum x":""} for this figure panel, where the "minimum x" value should be a float, calculated from the data values used to create the plot.
ANSWER: {'minimum': {'plot0': {'minimum x': -0.9039030182959754}}}
QUESTION: You are a helpful assistant that can analyze images.  How many gaussians were used to generate the data for the plot in the figure panel? Please format the output as a json as {"ngaussians":""} for this figure panel, where the "ngaussians" value should be an integer.
ANSWER: {'plot0': {'ngaussians': 2}}
QUESTION: You are a helpful assistant that can ana

In [16]:
axes

[[]]